In [1]:
# dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# read cancercsv into dataframe
cancer_df = pd.read_csv("static/csv/skin_map_incidence.csv")
cancer_df.head()

,State,Range,Rate
0,AK,9.5 to 19.0,14.9
1,AL,9.5 to 19.0,19.0
2,AR,19.1 to 21.9,20.0
3,AZ,19.1 to 21.9,20.6
4,CA,23.0 to 25.8,23.0


In [3]:
# read solar install csv into dataframe
solar_df = pd.read_csv("static/csv/openpv-export-201803141836.csv")
solar_df.head(2)

,state,date_installed,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,effic_1,cust_county,tracking,inv_model_clean,mod_cost_real,inv_cost_real,bos_powerclerk_real,permitting_real,3rdparty,Filters=maxdate=1420045199&mindate=1388552400
0,CO,01/01/2014,NaN,uir,13.58,NaN,80401,Government,NaN,1.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NY,01/01/2014,NaN,uir,306.60,NaN,11215,commercial,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
solar_targs_df = solar_df.iloc[:,[0, 1, 4, 6, 9]]
solar_targs_df.columns = ["state", "install_Data", "size_kw", "zip", "cost_per_Watt"]
solar_targs_df.head(2)

,state,install_Data,size_kw,zip,cost_per_Watt
0,CO,01/01/2014,13.58,80401,1.62
1,NY,01/01/2014,306.60,11215,NaN


In [5]:
# check to see how deeply the nans permiate our data
solar_targs_df.count()


state            170862
install_Data     170862
size_kw          170862
zip              170862
cost_per_Watt     73212
dtype: int64

In [6]:
# read eia cost data into dataframe
eia_df = pd.read_csv("static/csv/eia_2014_scrape.csv")
eia_df = eia_df.iloc[:,[1,2]]
eia_df["State"] = ["AL","AK","AZ","AR","CA","CO","CT","DE","DC","FL","GA","HI","ID","IL","IN","IA","KS",
                   "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC",
                   "ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
# eia_df

In [7]:
# merge cancer and electric cost dfs
cande_df = pd.merge(cancer_df, eia_df, on="State", how="outer")
cande_df.columns = ["state", "range", "rate", "name", "average retail price (cents/kWh)"]
electric_cancer = cande_df[["name","state","rate","average retail price (cents/kWh)"]]
electric_cancer.columns = ["state","sbbr","cancer_Rate","electric_Cost"]
#electric_cancer.head()
electric_cancer.to_csv("static/csv/electric_cost_cancer_rate.csv")

In [17]:
# merge solar and cande dfs
data_df = pd.merge(cande_df, solar_targs_df, on="state", how="outer")

# reset index to serve as id column
data_df = data_df.reset_index()
#data_df.head()

# rename columns and reorder columns
data_df.columns = ["id", "abbr", "c_range", "cancer_Rate", "state", "electric_Cost_cents_kWh)", "install_Date",
                  "size_kW", "zipcode", "cost_per_Watt"]
data_df = data_df[["id", "state", "abbr", "electric_Cost_cents_kWh)", "install_Date", "zipcode", "size_kW",
                 "cost_per_Watt", "c_range", "cancer_Rate"]]
data_df["size_kW"].fillna(0, inplace=True)
#data_df.head(10)

# push clean df to csv
data_df.to_csv("static/csv/clean_data.csv")

In [9]:
one = []

for i in data_df["size_kW"]:
    i == data_df["size_kW"]
    if 0 < i < 10:
        one.append(i)
    else :
        one.append(0)

two = []
for i in data_df["size_kW"]:
    i == data_df["size_kW"]
    if 10 <= i < 100:
        two.append(i)
    else :
        two.append(0)

three = []
for i in data_df["size_kW"]:
    i == data_df["size_kW"]
    if i >= 100:
        three.append(i)
    else :
        three.append(0)

data_df["size_one"] = one
data_df["size_two"] = two
data_df["size_three"] = three

#data_df.head(50)

In [10]:
# create df with total installed kW by state
installed = data_df.groupby(["state", "abbr"])["size_kW", "size_one", "size_two", "size_three"].sum()
installed_df = pd.DataFrame(installed)
installs_df = installed_df.reset_index()
# installs_df
installs_df.to_csv("static/csv/install_totals_state.csv")

In [11]:
install_groups = installs_df.drop("size_kW", axis=1)
install_groups.head()
install_groups.to_csv("static/csv/install_groups_state.csv")

In [12]:
installs_ecost = installs_df
installs_ecost["electric_cost"] = electric_cancer["electric_Cost"]
# installs_ecost
installs_ecost.to_csv("static/csv/installs_ecost.csv")

In [13]:
data_df.drop("size_one", axis=1, inplace=True)
data_df.drop("size_two", axis=1, inplace=True)
data_df.drop("size_three", axis=1, inplace=True)
#data_df.head()

In [14]:
# push data to sqlite db
engine = create_engine('sqlite:///clean_solar_data.db')
data_df.to_sql("data", con=engine, flavor="sqlite", if_exists="replace", index=False)
installs_df.to_sql("install_total", con=engine, flavor="sqlite", if_exists="replace", index=False)
electric_cancer.to_sql("electric_cancer", con=engine, flavor="sqlite", if_exists="replace", index=False)
installs_ecost.to_sql("installs_ecost", con=engine, flavor="sqlite", if_exists="replace", index=False)

C:\Users\ArneNewman\Anaconda3\envs\PythonData\lib\site-packages\pandas\io\sql.py:527: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


In [23]:
# test db completion
df = pd.read_sql_query("SELECT * FROM 'installs_ecost'", engine)
df.head()

,state,abbr,size_kW,size_one,size_two,size_three,electric_cost
0,Alabama,AL,0.000000e+00,0.000000,0.000000,0.000000,17.46
1,Alaska,AK,1.450000e+01,2.500000,12.000000,0.000000,9.27
2,Arizona,AZ,1.551172e+05,73347.394333,43593.181000,38176.638000,7.90
3,Arkansas,AR,0.000000e+00,0.000000,0.000000,0.000000,10.18
4,California,CA,1.173274e+06,510726.487028,124146.322401,538401.221577,15.15


In [60]:
#Setup datframes of top 10 sorted lists to include the state name, abbreviation, and metric; reset index

df_totalkw = df.sort_values(by=['size_kW'], ascending = False).head(n=10).iloc[:,0:3].reset_index(drop = True)
df_small= df.sort_values(by=['size_one'], ascending = False).head(n=10).loc[:,["state","abbr","size_one"]].reset_index(drop = True)
df_medium = df.sort_values(by=['size_two'], ascending = False).head(n=10).loc[:,["state","abbr","size_two"]].reset_index(drop = True)
df_large = df.sort_values(by=['size_three'], ascending = False).head(n=10).loc[:,["state","abbr","size_three"]].reset_index(drop = True)
df_cost = df.sort_values(by=['electric_cost'], ascending = False).head(n=10).loc[:,["state","abbr","electric_cost"]].reset_index(drop = True)


,state,abbr,size_three
0,California,CA,538401.221577
1,Massachusetts,MA,383379.211000
2,New Jersey,NJ,104205.158000
3,Indiana,IN,38930.490000
4,Arizona,AZ,38176.638000
5,Texas,TX,26903.400000
6,New York,NY,16135.550000
7,Ohio,OH,10479.295000
8,Nevada,NV,7774.500000
9,Vermont,VT,6250.400000


In [62]:
#push files to csv

df_totalkw.to_csv('/static/csv/total_kw_table.csv',index_label = "Rank")
df_small.to_csv('/static/csv/small_kw_table.csv',index_label = "Rank")
df_medium.to_csv('/static/csv/medium_kw_table.csv',index_label = "Rank")
df_large.to_csv('/static/csv/large_kw_table.csv',index_label = "Rank")
df_cost.to_csv('/static/csv/elec_cost_table.csv',index_label = "Rank")
